In [1]:
import sys
import time
sys.path.insert(0, '/home/anish/ASC_ML')

In [2]:
from ASC_ML import multiple_model_gen_v2 as multiple
from ASC_ML import dataframe_extractor as de
from ASC_ML import model_generation as model_gen
# from ASC_ML import model_gen_train_test as testing
from dask_ml.preprocessing import LabelEncoder
# from dask_ml.preprocessing import MinMaxScaler
from sklearn.preprocessing import MinMaxScaler

2022-06-18 19:26:04.142630: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [3]:
# m = multiple.Multiple_Model_Gen(max_no_layers = 2, model_per_batch = 10)

In [4]:
# import dask.dataframe as dd
# import pandas as pd
# data = dd.from_pandas(pd.Series(['a', 'a', 'b'], dtype='category'),
#                      npartitions=1)
# le = LabelEncoder()
# le.fit_transform(data).compute()
# print(type(le.fit_transform(data).compute()))

In [5]:
#For House Prediction Dataset https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data?select=train.csv
directory1 = "/home/anish/ASC-ML-EXP-DATASETS/LinReg-tabular/house-prices-advanced-regression-techniques/train.csv"


# returns list of dask dataframe [singular_df_x, singular_df_y] or [train_df_x, train_df_y, test_df_x, test_df_y]
dataset_list = de.DataframeExtractor_csv(directory1, label_names = ["SalePrice"]).df_list

print("Train Dataset X Columns")
dataset_list[0].head()

Reading single csv from /home/anish/ASC-ML-EXP-DATASETS/LinReg-tabular/house-prices-advanced-regression-techniques/train.csv
Train Dataset X Columns


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1.0,60.0,RL,65.0,8450.0,Pave,NaN,Reg,Lvl,AllPub,...,0.0,0.0,NaN,NaN,NaN,0.0,2.0,2008.0,WD,Normal
1,2.0,20.0,RL,80.0,9600.0,Pave,NaN,Reg,Lvl,AllPub,...,0.0,0.0,NaN,NaN,NaN,0.0,5.0,2007.0,WD,Normal
2,3.0,60.0,RL,68.0,11250.0,Pave,NaN,IR1,Lvl,AllPub,...,0.0,0.0,NaN,NaN,NaN,0.0,9.0,2008.0,WD,Normal
3,4.0,70.0,RL,60.0,9550.0,Pave,NaN,IR1,Lvl,AllPub,...,0.0,0.0,NaN,NaN,NaN,0.0,2.0,2006.0,WD,Abnorml
4,5.0,60.0,RL,84.0,14260.0,Pave,NaN,IR1,Lvl,AllPub,...,0.0,0.0,NaN,NaN,NaN,0.0,12.0,2008.0,WD,Normal


In [6]:
dataset_list[1].head()

,SalePrice
0,208500.0
1,181500.0
2,223500.0
3,140000.0
4,250000.0


In [7]:
train_x = dataset_list[0].copy()

In [8]:
train_x = train_x.fillna(0)

In [9]:
train_x.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1.0,60.0,RL,65.0,8450.0,Pave,0,Reg,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,2.0,2008.0,WD,Normal
1,2.0,20.0,RL,80.0,9600.0,Pave,0,Reg,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,5.0,2007.0,WD,Normal
2,3.0,60.0,RL,68.0,11250.0,Pave,0,IR1,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,9.0,2008.0,WD,Normal
3,4.0,70.0,RL,60.0,9550.0,Pave,0,IR1,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,2.0,2006.0,WD,Abnorml
4,5.0,60.0,RL,84.0,14260.0,Pave,0,IR1,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,12.0,2008.0,WD,Normal


In [10]:
train_x['MSSubClass'] = train_x['MSSubClass'].apply(str)

train_x['OverallCond'] = train_x['OverallCond'].astype(str)

train_x['YrSold'] = train_x['YrSold'].astype(str)
train_x['MoSold'] = train_x['MoSold'].astype(str)
train_x.head()

/home/anish/anaconda3/envs/tfgpu/lib/python3.9/site-packages/dask/dataframe/core.py:3946: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('MSSubClass', 'object'))

  warnings.warn(meta_warning(meta))


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1.0,60.0,RL,65.0,8450.0,Pave,0,Reg,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,2.0,2008.0,WD,Normal
1,2.0,20.0,RL,80.0,9600.0,Pave,0,Reg,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,5.0,2007.0,WD,Normal
2,3.0,60.0,RL,68.0,11250.0,Pave,0,IR1,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,9.0,2008.0,WD,Normal
3,4.0,70.0,RL,60.0,9550.0,Pave,0,IR1,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,2.0,2006.0,WD,Abnorml
4,5.0,60.0,RL,84.0,14260.0,Pave,0,IR1,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,12.0,2008.0,WD,Normal


In [11]:
# a = train_x['LotShape']

In [12]:
# le = LabelEncoder() 
# le.fit_transform(a).compute()
# train_x['LotShape'] = le.fit_transform(train_x['LotShape'])

In [13]:
# train_x.head()

In [14]:
def keyval(allkeys):
    i = 0
    keyvalpairs = list()
    for key in allkeys:
        keyvalpairs.append((key,i))
        i+=1
    return dict(keyvalpairs)

In [15]:
# cols = ['FireplaceQu', 'LotShape']
cols = ['FireplaceQu', 'BsmtQual', 'BsmtCond', 'GarageQual', 'GarageCond', 
        'ExterQual', 'ExterCond','HeatingQC', 'PoolQC', 'KitchenQual', 'BsmtFinType1', 
        'BsmtFinType2', 'Functional', 'Fence', 'BsmtExposure', 'GarageFinish', 'LandSlope',
        'LotShape', 'PavedDrive', 'Street', 'Alley', 'CentralAir', 'MSSubClass', 'OverallCond', 
        'YrSold', 'MoSold']
keys = {}
for col in cols:
    key = train_x[col].unique()
    keyvalpairs = keyval(key)
    keys.update({col:keyvalpairs})


In [16]:
keys

{'FireplaceQu': {0: 0, 'TA': 1, 'Gd': 2, 'Fa': 3, 'Ex': 4, 'Po': 5},
 'BsmtQual': {'Gd': 0, 'TA': 1, 'Ex': 2, 0: 3, 'Fa': 4},
 'BsmtCond': {'TA': 0, 'Gd': 1, 0: 2, 'Fa': 3, 'Po': 4},
 'GarageQual': {'TA': 0, 'Fa': 1, 'Gd': 2, 0: 3, 'Ex': 4, 'Po': 5},
 'GarageCond': {'TA': 0, 'Fa': 1, 0: 2, 'Gd': 3, 'Po': 4, 'Ex': 5},
 'ExterQual': {'Gd': 0, 'TA': 1, 'Ex': 2, 'Fa': 3},
 'ExterCond': {'TA': 0, 'Gd': 1, 'Fa': 2, 'Po': 3, 'Ex': 4},
 'HeatingQC': {'Ex': 0, 'Gd': 1, 'TA': 2, 'Fa': 3, 'Po': 4},
 'PoolQC': {0: 0, 'Ex': 1, 'Fa': 2, 'Gd': 3},
 'KitchenQual': {'Gd': 0, 'TA': 1, 'Ex': 2, 'Fa': 3},
 'BsmtFinType1': {'GLQ': 0,
  'ALQ': 1,
  'Unf': 2,
  'Rec': 3,
  'BLQ': 4,
  0: 5,
  'LwQ': 6},
 'BsmtFinType2': {'Unf': 0,
  'BLQ': 1,
  0: 2,
  'ALQ': 3,
  'Rec': 4,
  'LwQ': 5,
  'GLQ': 6},
 'Functional': {'Typ': 0,
  'Min1': 1,
  'Maj1': 2,
  'Min2': 3,
  'Mod': 4,
  'Maj2': 5,
  'Sev': 6},
 'Fence': {0: 0, 'MnPrv': 1, 'GdWo': 2, 'GdPrv': 3, 'MnWw': 4},
 'BsmtExposure': {'No': 0, 'Gd': 1, 'Mn': 2, '

In [17]:
train_X = train_x.replace(keys).copy()

In [18]:
train_X['FireplaceQu'].unique().compute()

0    0
1    1
2    2
3    3
4    4
5    5
Name: FireplaceQu, dtype: int64

In [19]:
train_X.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1.0,0,RL,65.0,8450.0,0,0,0,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,0,0,WD,Normal
1,2.0,1,RL,80.0,9600.0,0,0,0,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,1,1,WD,Normal
2,3.0,0,RL,68.0,11250.0,0,0,1,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,2,0,WD,Normal
3,4.0,2,RL,60.0,9550.0,0,0,1,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,0,2,WD,Abnorml
4,5.0,0,RL,84.0,14260.0,0,0,1,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,3,0,WD,Normal


In [20]:
cols_to_remove = []
for col in train_X.columns:
#     print(type(col))
    try:
        a = train_X[col].astype(float).compute()
    except ValueError:
#         print('Couldn\'t covert %s to float' % col)
        cols_to_remove.append(col)
        pass

# keep only the columns in df that do not contain string
train_X = train_X[[col for col in train_X.columns if col not in cols_to_remove]]

In [21]:
print(cols_to_remove)

['MSZoning', 'LandContour', 'Utilities', 'LotConfig', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'Foundation', 'Heating', 'Electrical', 'GarageType', 'MiscFeature', 'SaleType', 'SaleCondition']


In [22]:
train_X = train_X.drop(["Id"], axis = 1)

In [23]:
# tsklearn.preprocessing.MinMaxScalerrain_X.head()

In [24]:
len(train_X.columns)

58

In [25]:
len(dataset_list[1].columns)

1

In [26]:
scaler_x = MinMaxScaler()
scaler_x.fit(train_X)
train_X_scaled = scaler_x.transform(train_X)

scaler_y = MinMaxScaler()
scaler_y.fit(dataset_list[1])
train_Y_scaled = scaler_y.transform(dataset_list[1])

In [27]:
# col = train_X.columns.to_list()
# scaler_d = MinMaxScaler()
# train_X.columns = list(train_X.columns)
# scaled_train_X = scaler_d.fit_transform(train_X[col])

In [28]:
train_X_scaled
# train_Y_scaled

array([[0.        , 0.20766773, 0.0334198 , ..., 0.        , 0.        ,
        0.        ],
       [0.07142857, 0.25559105, 0.03879502, ..., 0.        , 0.09090909,
        0.25      ],
       [0.        , 0.2172524 , 0.04650728, ..., 0.        , 0.18181818,
        0.        ],
       ...,
       [0.14285714, 0.21086262, 0.03618687, ..., 0.16129032, 0.09090909,
        1.        ],
       [0.07142857, 0.2172524 , 0.03934189, ..., 0.        , 0.63636364,
        1.        ],
       [0.07142857, 0.23961661, 0.04037019, ..., 0.        , 1.        ,
        0.        ]])

In [29]:
train_Y_scaled

array([[0.24107763],
       [0.20358284],
       [0.26190807],
       ...,
       [0.321622  ],
       [0.14890293],
       [0.15636717]])

In [30]:
m = multiple.Multiple_Model_Gen_V2(train_X_scaled, train_Y_scaled, 50, 128, input_shape = 58, model_per_batch = 5)

In [31]:
m.get_model_confs()

[[(16, 16), (16, 32), (16, 64), (16, 128), (16, 196), (16, 256), (32, 16), (32, 32), (32, 64), (32, 128), (32, 196), (32, 256), (64, 16), (64, 32), (64, 64), (64, 128), (64, 196), (64, 256), (128, 16), (128, 32), (128, 64), (128, 128), (128, 196), (128, 256), (196, 16), (196, 32), (196, 64), (196, 128), (196, 196), (196, 256), (256, 16), (256, 32), (256, 64), (256, 128), (256, 196), (256, 256)], [(16, 16, 16), (16, 16, 32), (16, 16, 64), (16, 16, 128), (16, 16, 196), (16, 16, 256), (16, 32, 16), (16, 32, 32), (16, 32, 64), (16, 32, 128), (16, 32, 196), (16, 32, 256), (16, 64, 16), (16, 64, 32), (16, 64, 64), (16, 64, 128), (16, 64, 196), (16, 64, 256), (16, 128, 16), (16, 128, 32), (16, 128, 64), (16, 128, 128), (16, 128, 196), (16, 128, 256), (16, 196, 16), (16, 196, 32), (16, 196, 64), (16, 196, 128), (16, 196, 196), (16, 196, 256), (16, 256, 16), (16, 256, 32), (16, 256, 64), (16, 256, 128), (16, 256, 196), (16, 256, 256), (32, 16, 16), (32, 16, 32), (32, 16, 64), (32, 16, 128), (32

In [32]:
m.model_confs

[[['', 58, 2, 'relu', {'layer1': 16, 'layer2': 16}, [1, None]],
  ['', 58, 2, 'relu', {'layer1': 16, 'layer2': 32}, [1, None]],
  ['', 58, 2, 'relu', {'layer1': 16, 'layer2': 64}, [1, None]],
  ['', 58, 2, 'relu', {'layer1': 16, 'layer2': 128}, [1, None]],
  ['', 58, 2, 'relu', {'layer1': 16, 'layer2': 196}, [1, None]]],
 [['', 58, 2, 'relu', {'layer1': 16, 'layer2': 256}, [1, None]],
  ['', 58, 2, 'relu', {'layer1': 32, 'layer2': 16}, [1, None]],
  ['', 58, 2, 'relu', {'layer1': 32, 'layer2': 32}, [1, None]],
  ['', 58, 2, 'relu', {'layer1': 32, 'layer2': 64}, [1, None]],
  ['', 58, 2, 'relu', {'layer1': 32, 'layer2': 128}, [1, None]]],
 [['', 58, 2, 'relu', {'layer1': 32, 'layer2': 196}, [1, None]],
  ['', 58, 2, 'relu', {'layer1': 32, 'layer2': 256}, [1, None]],
  ['', 58, 2, 'relu', {'layer1': 64, 'layer2': 16}, [1, None]],
  ['', 58, 2, 'relu', {'layer1': 64, 'layer2': 32}, [1, None]],
  ['', 58, 2, 'relu', {'layer1': 64, 'layer2': 64}, [1, None]]],
 [['', 58, 2, 'relu', {'layer1'

In [ ]:
# m.get_all_models()
m.train_models()

2022-06-18 19:26:16.430470: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-06-18 19:26:16.432938: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-06-18 19:26:16.515787: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-18 19:26:16.516134: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 2060 computeCapability: 7.5
coreClock: 1.695GHz coreCount: 30 deviceMemorySize: 5.79GiB deviceMemoryBandwidth: 312.97GiB/s
2022-06-18 19:26:16.516178: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-06-18 19:26:16.518633: I tensorflow/stream_executor/platform/de

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_layer_dense_16_16 (InputL [(None, 58)]         0                                            
__________________________________________________________________________________________________
input_layer_dense_16_32 (InputL [(None, 58)]         0                                            
__________________________________________________________________________________________________
input_layer_dense_16_64 (InputL [(None, 58)]         0                                            
__________________________________________________________________________________________________
input_layer_dense_16_128 (Input [(None, 58)]         0                                            
______________________________________________________________________________________________

2022-06-18 19:26:17.902563: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10


12/12 [==============================] - 0s 3ms/step - loss: 0.0426 - output_layer_dense_16_16_loss: 0.0091 - output_layer_dense_16_32_loss: 0.0097 - output_layer_dense_16_64_loss: 0.0057 - output_layer_dense_16_128_loss: 0.0098 - output_layer_dense_16_196_loss: 0.0083 - output_layer_dense_16_16_mean_absolute_error: 0.0721 - output_layer_dense_16_32_mean_absolute_error: 0.0737 - output_layer_dense_16_64_mean_absolute_error: 0.0552 - output_layer_dense_16_128_mean_absolute_error: 0.0752 - output_layer_dense_16_196_mean_absolute_error: 0.0665
Epoch 3/50
12/12 [==============================] - 0s 3ms/step - loss: 0.0349 - output_layer_dense_16_16_loss: 0.0075 - output_layer_dense_16_32_loss: 0.0078 - output_layer_dense_16_64_loss: 0.0052 - output_layer_dense_16_128_loss: 0.0078 - output_layer_dense_16_196_loss: 0.0067 - output_layer_dense_16_16_mean_absolute_error: 0.0589 - output_layer_dense_16_32_mean_absolute_error: 0.0625 - output_layer_dense_16_64_mean_absolute_error: 0.0495 - outpu

12/12 [==============================] - 0s 3ms/step - loss: 0.0115 - output_layer_dense_16_16_loss: 0.0026 - output_layer_dense_16_32_loss: 0.0025 - output_layer_dense_16_64_loss: 0.0020 - output_layer_dense_16_128_loss: 0.0025 - output_layer_dense_16_196_loss: 0.0019 - output_layer_dense_16_16_mean_absolute_error: 0.0328 - output_layer_dense_16_32_mean_absolute_error: 0.0344 - output_layer_dense_16_64_mean_absolute_error: 0.0282 - output_layer_dense_16_128_mean_absolute_error: 0.0327 - output_layer_dense_16_196_mean_absolute_error: 0.0289
Epoch 17/50
12/12 [==============================] - 0s 3ms/step - loss: 0.0132 - output_layer_dense_16_16_loss: 0.0028 - output_layer_dense_16_32_loss: 0.0030 - output_layer_dense_16_64_loss: 0.0023 - output_layer_dense_16_128_loss: 0.0028 - output_layer_dense_16_196_loss: 0.0022 - output_layer_dense_16_16_mean_absolute_error: 0.0334 - output_layer_dense_16_32_mean_absolute_error: 0.0355 - output_layer_dense_16_64_mean_absolute_error: 0.0299 - outp

...

12/12 [==============================] - 0s 3ms/step - loss: 0.0027 - output_layer_dense_32_196_256_loss: 4.0842e-04 - output_layer_dense_32_256_16_loss: 5.9771e-04 - output_layer_dense_32_256_32_loss: 5.6722e-04 - output_layer_dense_32_256_64_loss: 6.9776e-04 - output_layer_dense_32_256_128_loss: 4.4823e-04 - output_layer_dense_32_196_256_mean_absolute_error: 0.0140 - output_layer_dense_32_256_16_mean_absolute_error: 0.0178 - output_layer_dense_32_256_32_mean_absolute_error: 0.0170 - output_layer_dense_32_256_64_mean_absolute_error: 0.0180 - output_layer_dense_32_256_128_mean_absolute_error: 0.0151
Epoch 41/50
12/12 [==============================] - 0s 4ms/step - loss: 0.0024 - output_layer_dense_32_196_256_loss: 3.9618e-04 - output_layer_dense_32_256_16_loss: 4.6040e-04 - output_layer_dense_32_256_32_loss: 5.4687e-04 - output_layer_dense_32_256_64_loss: 6.2538e-04 - output_layer_dense_32_256_128_loss: 4.1100e-04 - output_layer_dense_32_196_256_mean_absolute_error: 0.0137 - output_la

12/12 [==============================] - 1s 3ms/step - loss: 0.2394 - output_layer_dense_32_256_196_loss: 0.0985 - output_layer_dense_32_256_256_loss: 0.0154 - output_layer_dense_64_16_16_loss: 0.0503 - output_layer_dense_64_16_32_loss: 0.0410 - output_layer_dense_64_16_64_loss: 0.0342 - output_layer_dense_32_256_196_mean_absolute_error: 0.2563 - output_layer_dense_32_256_256_mean_absolute_error: 0.0954 - output_layer_dense_64_16_16_mean_absolute_error: 0.1984 - output_layer_dense_64_16_32_mean_absolute_error: 0.1628 - output_layer_dense_64_16_64_mean_absolute_error: 0.1465
Epoch 2/50
12/12 [==============================] - 0s 3ms/step - loss: 0.0463 - output_layer_dense_32_256_196_loss: 0.0099 - output_layer_dense_32_256_256_loss: 0.0053 - output_layer_dense_64_16_16_loss: 0.0097 - output_layer_dense_64_16_32_loss: 0.0116 - output_layer_dense_64_16_64_loss: 0.0097 - output_layer_dense_32_256_196_mean_absolute_error: 0.0690 - output_layer_dense_32_256_256_mean_absolute_error: 0.0539 -

12/12 [==============================] - 0s 4ms/step - loss: 0.0113 - output_layer_dense_32_256_196_loss: 0.0022 - output_layer_dense_32_256_256_loss: 0.0014 - output_layer_dense_64_16_16_loss: 0.0029 - output_layer_dense_64_16_32_loss: 0.0029 - output_layer_dense_64_16_64_loss: 0.0021 - output_layer_dense_32_256_196_mean_absolute_error: 0.0296 - output_layer_dense_32_256_256_mean_absolute_error: 0.0238 - output_layer_dense_64_16_16_mean_absolute_error: 0.0335 - output_layer_dense_64_16_32_mean_absolute_error: 0.0338 - output_layer_dense_64_16_64_mean_absolute_error: 0.0293
Epoch 15/50
12/12 [==============================] - 0s 4ms/step - loss: 0.0090 - output_layer_dense_32_256_196_loss: 0.0017 - output_layer_dense_32_256_256_loss: 0.0012 - output_layer_dense_64_16_16_loss: 0.0022 - output_layer_dense_64_16_32_loss: 0.0022 - output_layer_dense_64_16_64_loss: 0.0017 - output_layer_dense_32_256_196_mean_absolute_error: 0.0276 - output_layer_dense_32_256_256_mean_absolute_error: 0.0245 

12/12 [==============================] - 0s 4ms/step - loss: 0.0055 - output_layer_dense_32_256_196_loss: 9.7383e-04 - output_layer_dense_32_256_256_loss: 4.8435e-04 - output_layer_dense_64_16_16_loss: 0.0015 - output_layer_dense_64_16_32_loss: 0.0015 - output_layer_dense_64_16_64_loss: 0.0011 - output_layer_dense_32_256_196_mean_absolute_error: 0.0220 - output_layer_dense_32_256_256_mean_absolute_error: 0.0161 - output_layer_dense_64_16_16_mean_absolute_error: 0.0253 - output_layer_dense_64_16_32_mean_absolute_error: 0.0251 - output_layer_dense_64_16_64_mean_absolute_error: 0.0229
Epoch 28/50
12/12 [==============================] - 0s 3ms/step - loss: 0.0058 - output_layer_dense_32_256_196_loss: 0.0011 - output_layer_dense_32_256_256_loss: 6.6474e-04 - output_layer_dense_64_16_16_loss: 0.0015 - output_layer_dense_64_16_32_loss: 0.0015 - output_layer_dense_64_16_64_loss: 0.0011 - output_layer_dense_32_256_196_mean_absolute_error: 0.0214 - output_layer_dense_32_256_256_mean_absolute_er

12/12 [==============================] - 0s 3ms/step - loss: 0.0035 - output_layer_dense_32_256_196_loss: 5.2726e-04 - output_layer_dense_32_256_256_loss: 2.6716e-04 - output_layer_dense_64_16_16_loss: 0.0010 - output_layer_dense_64_16_32_loss: 0.0010 - output_layer_dense_64_16_64_loss: 6.6212e-04 - output_layer_dense_32_256_196_mean_absolute_error: 0.0165 - output_layer_dense_32_256_256_mean_absolute_error: 0.0122 - output_layer_dense_64_16_16_mean_absolute_error: 0.0220 - output_layer_dense_64_16_32_mean_absolute_error: 0.0214 - output_layer_dense_64_16_64_mean_absolute_error: 0.0186    
Epoch 41/50
12/12 [==============================] - 0s 3ms/step - loss: 0.0040 - output_layer_dense_32_256_196_loss: 5.5912e-04 - output_layer_dense_32_256_256_loss: 2.9849e-04 - output_layer_dense_64_16_16_loss: 0.0012 - output_layer_dense_64_16_32_loss: 0.0013 - output_layer_dense_64_16_64_loss: 6.6794e-04 - output_layer_dense_32_256_196_mean_absolute_error: 0.0165 - output_layer_dense_32_256_256_